In [1]:
import pandas as pd
from pymongo import MongoClient
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import collections
%matplotlib inline

# Functions to get data from database

In [2]:

def mid_and_imbalance(value):
    value['mid_price'] = (value['bids_price'][0]+value['asks_price'][0])/2
    l = np.sum(value['asks_price']*value['asks_volume']) + np.sum(value['bids_price']*value['bids_volume'])
    totvol = (np.sum(value['asks_volume'])+ np.sum(value['bids_volume']))
    value['imbalance'] = l/totvol - value['mid_price']
    return value

def flatten(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, collections.MutableMapping):
            items.extend(flatten(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

def get_order_depth(pair:str, start_date:datetime, num_items):
    """
    pair : 'EOSUSD', ' # Eos
            BCHUSD',   # bth, bitcoin cash
            'XXLMZUSD', # stellar lumen
             'XXMRZUSD', # monero
             'XXRPZUSD', # ripple
             'XETHZUSD', # ethereum
             'XLTCZUSD', # lightcoin
             'XETCZUSD'  # ethereum classic
    """
    
    print('Connecting To MongoDB')
    conn = MongoClient('userver', 27017)
    db = conn.ticker3_db
    # make query
    d = datetime(year=2019,month=1,day=1)
    cursor  = db['kraken_orderdepth'].find({"timestamp":{"$gte":start_date}, "data.pair":pair}).limit(num_items)
    print(f'found {cursor.count(with_limit_and_skip=True)} items')
    
    print('flatten array')
    # flatten the data and make a list
    l = list()
    for data in cursor:
        d = flatten(data['data'])
        d['timestamp'] = data['timestamp']
        l.append(d)
        
    # create dataframe and convert strings to floats
    df = pd.DataFrame(l)
    df['asks_price'] = df['asks_price'].apply(np.asfarray, np.float)
    df['asks_volume'] = df['asks_volume'].apply(np.asfarray, np.float)
    df['bids_price'] = df['bids_price'].apply(np.asfarray, np.float)
    df['bids_volume'] = df['bids_volume'].apply(np.asfarray, np.float)
    
    print('Adding extra columns')
    df = df.apply(mid_and_imbalance, axis=1)
    print('Done')
    
    return df

In [3]:
df_xlm = get_order_depth(pair="XXLMZUSD", start_date=datetime(year=2019,month=1,day=1), num_items=20000)
df_xlm.head()

Connecting To MongoDB
found 20000 items
flatten array
Adding extra columns
Done


,asks_price,asks_timestamp,asks_volume,bids_price,bids_timestamp,bids_volume,pair,timestamp,mid_price,imbalance
0,"[0.109698, 0.109813, 0.109836, 0.109952, 0.109...","[1546297231, 1546297229, 1546297129, 154629723...","[450.0, 60417.471, 68.261, 104.588, 4000.0, 14...","[0.109241, 0.109236, 0.109235, 0.109114, 0.109...","[1546297225, 1546297231, 1546297215, 154629721...","[300.0, 300.0, 207.675, 7287.673, 103389.31, 7...",XXLMZUSD,2019-01-01 00:00:32.860,0.109469,-0.000083
1,"[0.109722, 0.109724, 0.109727, 0.109813, 0.109...","[1546297291, 1546297284, 1546297250, 154629722...","[370.0, 440.0, 1006.373, 60417.471, 68.261, 12...","[0.109262, 0.109129, 0.109124, 0.109123, 0.109...","[1546297282, 1546297291, 1546297289, 154629728...","[207.675, 300.0, 7287.673, 300.0, 103389.31, 7...",XXLMZUSD,2019-01-01 00:01:32.186,0.109492,-0.000514
2,"[0.109734, 0.109739, 0.109741, 0.109813, 0.109...","[1546297345, 1546297351, 1546297319, 154629722...","[335.0, 450.0, 1006.373, 60417.471, 86.394, 86...","[0.109282, 0.109277, 0.109275, 0.109128, 0.109...","[1546297347, 1546297351, 1546297343, 154629731...","[300.0, 300.0, 2836.202, 7287.673, 103389.31, ...",XXLMZUSD,2019-01-01 00:02:32.371,0.109508,-0.001875
3,"[0.109808, 0.10981, 0.109813, 0.10987, 0.10994...","[1546297411, 1546297405, 1546297229, 154629739...","[370.0, 440.0, 60417.471, 1007.387, 86.394, 86...","[0.109308, 0.109303, 0.109128, 0.109097, 0.109...","[1546297412, 1546297410, 1546297314, 154629713...","[300.0, 8636.202, 7287.673, 103389.31, 76763.5...",XXLMZUSD,2019-01-01 00:03:32.921,0.109558,-0.001254
4,"[0.109806, 0.109813, 0.10987, 0.109949, 0.1099...","[1546297468, 1546297426, 1546297390, 154629732...","[335.0, 59298.637, 1007.387, 86.394, 86.396, 4...","[0.109327, 0.109325, 0.10932, 0.109134, 0.1090...","[1546297469, 1546297467, 1546297465, 154629746...","[300.0, 3037.673, 5000.0, 4250.0, 103389.31, 1...",XXLMZUSD,2019-01-01 00:04:33.127,0.109566,-0.001076
